## Rating products

In [13]:
#Rating products - derste anlatılanların denemesi 

import pandas as pd 
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [14]:
# (50+ Saat) Python A-Z™: Veri Bilimi ve Machine Learning
# Puan: 4.8 (4.764925)
# Toplam Puan: 4611
# Puan Yüzdeleri: 75, 20, 4, 1, <1
# Yaklaşık Sayısal Karşılıkları: 3458, 922, 184, 46, 6

In [15]:
df = pd.read_csv('course_reviews.csv')
df.head(3)

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000


In [16]:
df.shape
#kursa ait değerlendirme sayısını öğrenmek için shape metodundan yararlanabiliriz

(4323, 6)

In [17]:
df.columns

Index(['Rating', 'Timestamp', 'Enrolled', 'Progress', 'Questions Asked', 'Questions Answered'], dtype='object')

In [18]:
df['Rating'].value_counts()
#puanlamanın dağılımını incelemek için

5.00000    3267
4.50000     475
4.00000     383
3.50000      96
3.00000      62
1.00000      15
2.00000      12
2.50000      11
1.50000       2
Name: Rating, dtype: int64

In [19]:
#soru soran kullanıcıların dağılımlarını incelemek için
df['Questions Asked'].value_counts()
#örneğin 11 adet soru soran kullanıcı sayısı 2

0.00000     3867
1.00000      276
2.00000       80
3.00000       43
4.00000       15
5.00000       13
6.00000        9
8.00000        5
9.00000        3
14.00000       2
11.00000       2
7.00000        2
10.00000       2
15.00000       2
22.00000       1
12.00000       1
Name: Questions Asked, dtype: int64

In [20]:
df.groupby('Questions Asked').agg({'Questions Asked':'count',
                                   'Rating': 'mean'})
#yorumlama --> hiç soru sormayan kullanıcıların vermiş olduğu puanların ortlamaası 4,77'dir.

,Questions Asked,Rating
Questions Asked,,
0.00000,3867,4.76519
1.00000,276,4.74094
2.00000,80,4.80625
3.00000,43,4.74419
4.00000,15,4.83333
5.00000,13,4.65385
6.00000,9,5.00000
7.00000,2,4.75000
8.00000,5,4.90000


Kursa Ait Puanların Ortalaması

In [21]:
df['Rating'].mean()

4.764284061993986

Dikkat: Bu kursa verilen puanların sadece ortalamasını alarak kursun başarısını yorumlamak doğru olamayabilir. "Memnuniyet trendi"ni kaçırmamıza sebebiyet verebilir, dolayısıyla başka yöntemlere başvurulması gerekebilir.

## Puan Zamanlarına Göre Ağırlıklı Ortalama 
>Puan verilen tarihi, ortalama puan hesaplarken düşünmeli miyiz? Düşünürsek nasıl bir hesaplama yapmalıyız?

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rating              4323 non-null   float64
 1   Timestamp           4323 non-null   object 
 2   Enrolled            4323 non-null   object 
 3   Progress            4323 non-null   float64
 4   Questions Asked     4323 non-null   float64
 5   Questions Answered  4323 non-null   float64
dtypes: float64(4), object(2)
memory usage: 202.8+ KB


Timestamp sütunu object tipindedir, bu sütunu datetime tipine çevirmeliyiz.

In [23]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rating              4323 non-null   float64       
 1   Timestamp           4323 non-null   datetime64[ns]
 2   Enrolled            4323 non-null   object        
 3   Progress            4323 non-null   float64       
 4   Questions Asked     4323 non-null   float64       
 5   Questions Answered  4323 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 202.8+ KB


In [24]:
current_date = pd.to_datetime('2021-02-10 0:0:0')
#yorumların kaç günlük olduğu hesaplamak için sabit bir tarih belirlendi - veri seti eski olduğu için tarih de eski.

In [25]:
df['Days'] = (current_date - df['Timestamp']).dt.days

In [26]:
df.head(3)

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,Days
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000,4
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000,5
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000,5


In [27]:
#yorum yapma günü 30 günden az olan kayıtları filtreleme
df[df['Days'] <= 30]

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,Days
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000,4
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000,5
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000,5
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000,5
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000,5
5,4.00000,2021-02-04 12:42:36,2021-02-01 15:40:13,1.00000,0.00000,0.00000,5
6,5.00000,2021-02-04 12:25:30,2020-11-30 19:23:54,85.00000,0.00000,4.00000,5
7,4.50000,2021-02-04 11:13:15,2021-01-08 08:05:42,10.00000,0.00000,0.00000,5
8,5.00000,2021-02-04 08:59:53,2021-02-02 18:14:49,1.00000,0.00000,0.00000,5
9,5.00000,2021-02-03 22:40:04,2020-08-01 22:30:42,35.00000,0.00000,0.00000,6


In [28]:
df[df['Days'] <= 30].count()

Rating                194
Timestamp             194
Enrolled              194
Progress              194
Questions Asked       194
Questions Answered    194
Days                  194
dtype: int64

In [29]:
df.loc[df['Days'] <= 30, 'Rating'].mean()

4.775773195876289

In [30]:
#yorum gününün üzerinde 30 gün geçmiş ancak 90 günden az olan yorumların puan ortalaması
df.loc[(df['Days'] > 30) & (df['Days'] <= 90), 'Rating'].mean()

4.763833992094861

In [31]:
df.loc[(df['Days'] > 90) & (df['Days'] <= 180), 'Rating'].mean()

4.752503576537912

In [32]:
df.loc[df['Days'] > 180, 'Rating'].mean()

4.76641586867305

Yukarıdaki değerlere bakıldığında ortalama puanların artması bu kursa verilen puanların son 30 günde artış olduğunu göstermektedir.

puan zamanlarına göre ağırlıklı bir ortalama puan hesaplamak için fonksiyon tanımlayıp ortalama değer hesaplanabilir.

In [33]:
#dataframein istenilen sütunlara sahip olmasına dikkat etmek gerekiyor. örn:days sütununu sonradan ekledik.
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    """
    Bu fonksiyon, verilen dataframe içindeki her bir sütunun değerlerini
    zaman bazlı ortalama hesaplar.

    Argümanlar:
    dataframe (pandas.DataFrame): Verilen dataframe.
    w1 (int): İlk zamanlı ortalama için göreceği gün sayısı.
    w2 (int): İkinci zamanlı ortalama için göreceği gün sayısı.
    w3 (int): Üçüncü zamanlı ortalama için göreceği gün sayısı.
    w4 (int): Dördüncü zamanlı ortalama için göreceği gün sayısı.
    """
    return dataframe.loc[dataframe['Days'] <= 30, 'Rating'].mean() * w1/100 + \
           dataframe.loc[(df['Days'] > 30) & (df['Days'] <= 90), 'Rating'].mean() * w2/100 + \
           dataframe.loc[(dataframe["Days"] > 90) & (dataframe["Days"] <= 180), "Rating"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["Days"] > 180), "Rating"].mean() * w4 / 100

In [34]:
time_based_weighted_average(df)

4.765025682267194

## Kullanıcı Temelli Ağırlıklı Ortalama 
>Kullanıcının kurstaki ilerlemesine göre verdiği puanın ağırlıklandırması söz konusu olduğunda ne yapmalıyız? 

Burada kalite metriği ya da nitelik metriği 'Progress'tir.

In [35]:
df.groupby('Progress').agg({'Rating':'mean'})

,Rating
Progress,
0.00000,4.67391
1.00000,4.64269
2.00000,4.65476
3.00000,4.66355
4.00000,4.77733
5.00000,4.69821
6.00000,4.75510
7.00000,4.73256
8.00000,4.74194


Yukarıdaki sonuca bakıldığında kursun izleme oranları ile verilen puan arasında bir ilişki olduğu söylenebilir.

In [47]:
def user_based_weighted_average(dataframe, w1=22, w2=24, w3=26, w4=28):
    return dataframe.loc[dataframe['Progress'] <= 10, 'Rating'].mean() * w1/100 + \
           dataframe.loc[(dataframe['Progress'] > 10) & (dataframe['Progress'] <= 45), 'Rating'].mean() * w2/100 + \
           dataframe.loc[(dataframe['Progress'] > 45) & (dataframe['Progress'] <= 75), 'Rating'].mean() * w3/100 + \
           dataframe.loc[dataframe['Progress'] > 75, 'Rating'].mean() * w4/100 

In [48]:
user_based_weighted_average(df)

4.800257704672543

## Ağırlıklı Derecelendirme

In [49]:
def weighted_rating(dataframe, time_w=50, user_w=50):
    return time_based_weighted_average(dataframe) * time_w / 100 + user_based_weighted_average(dataframe) * user_w / 100

In [50]:
weighted_rating(df)

4.782641693469868